# Modello di ML per il riconoscimento di cifre scritte a mano

#### *(Comparazione delle performance di CPU, GPU e TPU)*

Progetto per l'esame di Calcolatori Elettronici e Reti di Calcolatori

I dati utilizzati nel seguente modello sono disponibili come dataset attraverso la API `tensorflow.keras.datasets` sotto la rispettiva licenza (come indicato [qui](https://keras.io/api/datasets/mnist/)).


## Inclusione librerie
Come prima cosa, bisogna importare le librerie [numpy](https://numpy.org), [tensorflow](https://www.tensorflow.org) e [tensorflow_datasets](https://www.tensorflow.org/datasets):

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import time # usato per ottenere misure accurate del tempo di esecuzione dei vari passi

Di seguito è definita una classe per la misura del tempo dei vari passi

In [ ]:
# la classe è strutturata in questo modo per funzionare con i with statement
class timer:
  def __init__(self, enter_fn=None, exit_fn=None):
    self.enter_fn = enter_fn
    self.exit_fn = exit_fn
  
  def __enter__(self):
    self.begin = time.perf_counter()
    self.end = self.begin
    if self.enter_fn:
      self.enter_fn(self.begin)
    return lambda: self.end - self.begin
  
  def __exit__(self, exc_type, exc_data, exc_tb):
    self.end = time.perf_counter()
    if self.exit_fn:
      self.exit_fn(self.begin, self.end)
  


A questo punto è opportuno scegliere alcune impostazioni per l'esecuzione, tra cui il <ins>**runtime**</ins> (selezionabile dalle impostazioni di Colab), il <ins>**numero di comparazioni di test**</ins> da eseguire alla fine (per verificare la correttezza delle previsioni del modello), il <ins>**numero di `epochs`**</ins> (ovvero il numero di iterazioni di allenamento) e la <ins>**batch_size**</ins> (ovvero la dimensione dei dati in input per ogni iterazione):

In [ ]:
#@title Impostazioni

#@markdown Numero di comparazioni di test:
comparison_count = 400 #@param {type:"slider", min:100, max:1000, step:10}
epoch_count = 10 #@param {type:"slider", min:1, max:50, step:1}
batch_size = 400 #@param {type:"slider", min:100, max:1000, step:50}
#@markdown Selezionare il parametro seguente se si desidera utilizzare delle GPU:
use_gpu = True #@param {type:"boolean"}

# riconoscimento runtime e selezione strategy adeguata
strategy = tf.distribute.get_strategy()
device_type = "CPU"

# https://stackoverflow.com/a/62729266
try:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.TPUStrategy(resolver)
  device_type = "TPU"
except ValueError:
  # https://stackoverflow.com/a/38019608
  device_list = tf.config.list_logical_devices("GPU")
  if use_gpu and len(device_list) > 0:
    strategy = tf.distribute.MirroredStrategy(device_list)
    device_type = "GPU"

print("Dispositivi: ", tf.config.list_logical_devices(device_type))


### Caricamento del dataset
In questo caso viene usato il dataset MNIST fornito dalla libreria `tensorflow_datasets` contenente tutti i dati necessari per questo modello.
I dati vengono caricati separatamente in due datasets contenenti rispettivamente delle immagini (con le relative label) per l'allenamento e per il testing della rete.

In [ ]:
# caricamento dei dati
# dividendo per 255 si ottiene un intervallo tra 0 e 1 per le intensità del grigio
# per ogni immagine (e lo si trasforma in float)
def scale(img, lab):
  img = tf.cast(img, tf.float32)
  img /= 255.0
  return img, lab

with timer() as load_time:
  train_data, test_data = tfds.load(
      'mnist', # nome del dataset da caricare
      split=['train', 'test'], # insiemi di dati da caricare (allenamento e testing)
      as_supervised=True, # carica anche le labels delle immagini
      try_gcs=True # cerca il dataset su bucket GCS in modo da ottimizzare l'accesso per le TPU
  )

  train_data = train_data.map(scale).batch(batch_size) # scala e separa in batch
  test_data = test_data.map(scale).batch(batch_size)


Ora che i dati sono stati caricati, è possibile passare alle operazioni sul modello.

## Creazione del modello
La creazione del modello viene effettuata mediante la API [Keras](https://keras.io):

In [ ]:
with timer() as compilation_time:
  with strategy.scope():
    # crea una rete a 3 layer:
    # 1 - input layer: trasforma l'immagine in un array
    # 2 - hidden layer: composto da 128 unità, applica la funzione di attivazione (relu) e ne bilancia i pesi
    # 3 - output layer: restituisce 10 valori contenenti le probabilità per ogni cifra
    model = tf.keras.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)), 
      tf.keras.layers.Dense(128, activation='relu'), 
      tf.keras.layers.Dense(10)
    ])

    # compilazione del modello
    model.compile(
        optimizer='sgd', 
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
        metrics=['sparse_categorical_accuracy'])
  

Il metodo `tf.keras.Sequential` effettua la creazione della rete, prendendo come argomenti una lista di layers specificata dall'utente:
 - `tf.keras.layers.Flatten` si occupa esclusivamente di trasformare le immagini (che sono array bidimensionali 28x28) in array unidimensionali a 784 ($28^{2}$) elementi;
 - `tf.keras.layers.Dense` crea un layer che esegue una specifica funzione di attivazione (in questo caso **ReLU** per il layer intermedio (ovvero un [rettificatore](https://it.wikipedia.org/wiki/Rettificatore_(reti_neurali)), $f(x) = max(0,x)$) e nessuna per il layer di uscita;
---
Il metodo `compile` effettua la compilazione del modello, usando i parametri specificati dall'utente:
 - `optimizer` indica la funzione usata per l'ottimizzazione dell'uscita della rete (in questo caso è usata la funzione *SGD* che rappresenta la *discesa stocastica del gradiente*, che esamina il gradiente di un numero ristretto di campioni dei dati di input e ne valuta l'incremento basandosi sui parametri interni);
 - `loss` indica la funzione di costo o obiettivo (*loss function*) che rappresenta l'errore dell'uscita della rete rispetto all'uscita desiderata da ottimizzare (la corrispondenza tra le labels, in questo caso);
 - `metrics` indica i parametri di diagnostica desiderati sull'apprendimento della rete;

### Allenamento del modello


In [ ]:
with timer() as training_time:
  model.fit(train_data, epochs=epoch_count)

Il metodo `fit` fa partire l'allenamento della rete, andando ad utilizzare i parametri in ingresso:
 - `train_data` rappresenta le immagini in input e le labels per la supervisione;
 - `epochs` specifica il numero di iterazioni da eseguire sui dati prima di terminare l'allenamento;

## Test dei risultati

### Valutazione degli input di test

In [ ]:
with timer() as testing_time:
  loss, accuracy = model.evaluate(test_data)
print("Precisione test: {acc} ({acc_perc:.2f}%)".format(acc=accuracy, acc_perc=accuracy*100))

Il metodo `evaluate` effettua i test e ritorna una coppia `loss, accuracy` che indicano rispettivamente l'errore e la precisione dei test.

### Comparazione delle previsioni
A questo punto non rimane che provare a passare alcuni dati (presi dagli input di test) e verificare la precisione delle previsioni 

In [ ]:
pred = tf.keras.Sequential([model, tf.keras.layers.Softmax()]).predict(test_data)

# caricamento delle labels di test per le comparazioni delle previsioni
# può essere sostituito da qualsiasi dato affine al modello
_, test_labels = tf.keras.datasets.mnist.load_data()[1]

successful = 0
for i in range(0, comparison_count):
  if np.argmax(pred[i]) == test_labels[i]:
    successful += 1
  
print("Risultato (previsioni giuste/totale): {successful}/{total}".format(successful=successful, total=comparison_count))
print("Precisione modello: {value:.2f}%".format(value=100*successful/comparison_count))

### Valutazione dei tempi
Ecco i tempi impiegati per le varie fasi:

In [ ]:
print("Dispositivo: " + device_type + " (" + str(len(tf.config.list_logical_devices(device_type))) + " unità)")
print("Tempo per il caricamento dei dati: " + "{t:.3f} s".format(t=load_time()))
print("Tempo per la compilazione del modello: " + "{t:.3f} s".format(t=compilation_time()))
print("Tempo per l'allenamento del modello: " + "{t:.3f} s".format(t=training_time()))
print("Tempo per il testing del modello: " + "{t:.3f} s".format(t=testing_time()))